Ever wanted a recursive agent? Now you can have one! 🤯

In [1]:
# This page will use the following imports:

from lasagna import Model, EventCallback, AgentRun
from lasagna import (
    recursive_extract_messages,
    override_system_prompt,
    noop_callback,
    extraction,
    parallel_runs,
)
from lasagna import known_models
from lasagna.tui import tui_input_loop

from pydantic import BaseModel, Field

import os
import asyncio

from dotenv import load_dotenv

We need to set up our "binder" (see the [quickstart guide](../quickstart.ipynb) for what this is).

In [2]:
load_dotenv()

if os.environ.get('ANTHROPIC_API_KEY'):
    print('Using Anthropic')
    binder = known_models.anthropic_claude_sonnet_4_binder

elif os.environ.get('OPENAI_API_KEY'):
    print('Using OpenAI')
    binder = known_models.openai_gpt_5_mini_binder

else:
    assert False, "Neither OPENAI_API_KEY nor ANTHROPIC_API_KEY is set! We need at least one to do this demo."

Using Anthropic


## Example: Recursive Task Breakdown

Let's build a planning agent. It will recursively break a complex task into subtasks (and those subtasks into subsubtasks, etc).

In [8]:
class Plan(BaseModel):
    thoughts: str = Field(description='your free-form thoughts')
    task_statement: str = Field(description='a concisely-stated task description')
    is_trivial: bool = Field(description="true if this task has a trivial solution (such as a simple math calculation, a simple fact lookup, etc); false if this task needs to be broken up into subtasks")
    sub_tasks: list[str] = Field(description="an ordered list of subtasks that need to be performed in order to complete the task statement (if not trivial); else an empty list (if trivial)")


@binder
async def planning_agent(
    model: Model,
    event_callback: EventCallback,
    prev_runs: list[AgentRun],
) -> AgentRun:
    messages = recursive_extract_messages(prev_runs, from_tools=False, from_extraction=False)
    messages = override_system_prompt(messages, "You plan the execution of a task by analyzing it, determining its complexities, and (if required) breaking it up into subtasks.")
    message, result = await model.extract(
        event_callback,
        messages = messages,
        extraction_type = Plan,
    )
    # TODO: add the recursive stuff
    return extraction('planning_agent', [message], result)

In [9]:
await tui_input_loop(planning_agent)   # type: ignore[top-level-await]

>  Hi


Plan({"thoughts": "The user just said \"Hi\" which is a simple greeting. This is not really a task that needs to be completed or broken down into subtasks - it's just a social interaction. I should treat this as a trivial case since there's no actual work or problem-solving involved.", "task_statement": "Respond to a greeting", "is_trivial": true, "sub_tasks": []})



>  Find all Disney movies with a rotten tomatoe score of 90 or more


Plan({"thoughts": "This task requires finding Disney movies with a Rotten Tomatoes score of 90% or higher. This is not a trivial task because it involves multiple steps: I need to identify what constitutes \"Disney movies\" (which could include different studios under Disney), find their Rotten Tomatoes scores, filter for those with 90% or higher, and compile a comprehensive list. This will require research and data gathering from multiple sources.", "task_statement": "Find all Disney movies with a Rotten Tomatoes score of 90% or higher", "is_trivial": false, "sub_tasks": ["Define the scope of 'Disney movies' (Walt Disney Animation Studios, Pixar, Disney live-action, etc.)","Research and compile a comprehensive list of Disney movies across different studios/divisions","Look up Rotten Tomatoes scores for each Disney movie","Filter the list to include only movies with scores of 90% or higher","Organize and present the final list with movie titles, release years, and Rotten Tomatoes score

>  exit
